# Lab05: Improving QR factorisation

In [ ]:
import numpy as np

## Exercise 1: Problems with QR factorisation

The Gram-Schmidt process is known to not be numerically stable. In practice, this means that the method can have problems which lead to loss of orthogonality through the process. In this exercise, we will explore this problem on a simple $2 \times 2$ problem.

For $\varepsilon > 0$, let $A_\varepsilon$ be the matrix given by
$$
A_\varepsilon = \begin{pmatrix}
1 & 1 + \varepsilon \\
1 & 1
\end{pmatrix}.
$$

There are three ways to judge the accuracy of a QR factorisation
$$
\begin{aligned}
\text{error}_1 &= \| A - QR \|_2 \\
\text{error}_2 &= \| Q^T Q - I_{n} \|_2 \\
\text{error}_3 &= \| R - \text{toupper}(R) \|_2,
\end{aligned}
$$
where $\| . \|_2$ is a norm on matrices than can be computed using `np.linalg.norm(...)` and $\text{toupper}$ is a function that takes a matrix to only its upper trianglar part which can be computer using `np.triu(...)`.

1. Create a table with the columns $\varepsilon$, $\text{error}_1$, $\text{error}_2$ and $\text{error}_3$ for $\varepsilon = 10^{-6}, 10^{-7}, \ldots, 10^{-16}$ using the classical Gram-Scmidt process from the lectures.
2. Explain why you see the results are not as good as you would like.

<div class="alert alert-block alert-danger">
<b>Portfolio exercise</b>
You should submit:

1. The code to generate the table.
2. The tables of results.
3. Your explanation of the results.
</div>

## Exercise 2: Improving the QR with Gram Schmidt approach

In this exercise, we will improve our implementation of
`gram_schmidt_eigen`.

Implement *one* of the following improvements and see how the change the
resulting performance of the algorithm in terms of robustness, accuracy
and/or efficiency.

1.  Following [Example
    7.3](https://comp2870-2526.github.io/linear-algebra-notes/src/lec07.html#exm-char),
    we have a good formula for how *shifting* a matrix changes the
    eigenvalues and eigenvectors of a matrix.

    For a shift value $\mu$, the key idea for the shifted algorithm is
    to shift the matrix before computing QR factorisation $$
     A - \mu I_n = QR,
     $$ and then undo the shift when updating $$
     A^{\text{new}} = RQ + \mu I_n.
     $$

    There are two suggestions form the literature to improve the speed
    of convergence.

    -   The first is to take the bottom right value of the matrix $A$ at
        each step $\mu = A_{nn}$. Note this value changes throughout the
        computation as we update $A$ in-place.
    -   The second is to use the *Wilkinson shift*. In this case, we
        compute the eigenvalues of the bottom-right $2 \times 2$
        submatrix of $A$ (using your code from Lab04) and then
        choose the eigenvalue closest to the bottom-right element of $A$
        as our value of $\mu$.

    Neither change affects how the matrix $V$ should be computed.

2.  We can also scale the matrix to be close to unit size. We again do
    this before the QR factorisation and then undo during reconstruction

    1.  Set $\alpha = \max_{ij} | A_{ij} |$ and
        $\tilde{A} = A / \alpha$.
    2.  Compute the QR factorisation of $\tilde{A} = QR$.
    3.  Update the $A$ as $$A^{\text{new}} = \alpha RQ.$$

    This change does not affect how the matrix $V$ should be computed.

Test your solution by finding the eigenvalues and eigenvectors of 10 randomly created symmetric matrices of sizes $n = 2, 4, 8, 16, 32$. Comment on what you find.

For the method for the lectures and each approach you implement, make a table of problem size, average (mean) time taken, average QR iterations and maximum error as implemented by the `test_accuracy_of_eigensolve` function.

How could you improve the variation of the method further in order to improve the convergence?

In [ ]:
from scipy.stats import special_ortho_group

# replicable random seed
np.random.seed(42)


def random_symmetric_matrix(n):
    # generate a random matrix
    S = special_ortho_group.rvs(n)
    D = np.diag(np.random.randint(1, 10, (n,)) / 2)
    A = S.T @ D @ S
    return A


def test_accuracy_of_eigensolve(A, eigenvalues, eigenvectors):
    """
    test accuracy of solution of eigenvalue problem
    """
    residuals = []
    for i in range(len(eigenvalues)):
        residual = np.linalg.norm(
            A @ eigenvectors[:, i] - eigenvalues[i] * eigenvectors[:, i]
        )
        residuals.append(residual)
    return max(residuals)

If you have time, implement one or more additional updates again
comparing using the test cases from the lecture notes.